In [ ]:
##How long would it take to process each object individually?

In [ ]:
import numpy as np

##Import mask
mask5=np.load('C:/Users/David Brown/Documents/Huang Lab/Exp0104_KDM5A_PHD2-fragments/Output/Well03_HeLa_PHD2-IDR-PHD3[W1625A]_3_Mask5.npy')

In [ ]:
import time

In [ ]:
import skimage
skimage.__version__

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

z=4

In [ ]:
##Check the mask
plt.imshow(mask5[z])
plt.axis('off')
plt.title('Well03_HeLa_PHD2-IDR-PHD3[W1625A]_3')

In [ ]:
##Try Labeling objects
import mahotas as mh

labeled_mask5, nr_objects=mh.label(mask5)
print('{} objects were detected.'.format(nr_objects))

In [ ]:
##Remove small objects
from skimage import morphology

min_vol = 8000

#from skimage.morphology import remove_small_objects
real=morphology.remove_small_objects(labeled_mask5, min_vol)

In [ ]:
##Count the labels, Reset counts (Maybe Randomize?), Find objects
relabeled, n_left = mh.labeled.relabel(real)
print('After filtering and relabeling, there are {} nuclei left.'.format(n_left))

In [ ]:
plt.imshow(relabeled[z])
plt.axis('off')
plt.title('3D Labeled Mask')

In [ ]:
import mahotas as mh
from skimage import morphology

##
def labelNuceli(mask, min_vol = 8000):
#By default min_vol = 8000
    
    labeled_mask, nr_objects=mh.label(mask)
    print('{} objects were detected.'.format(nr_objects))
    
    real=morphology.remove_small_objects(labeled_mask, min_vol)
    
    #Reset counts  and count the labels, 
    relabeled, n_left = mh.labeled.relabel(real)
    print('After filtering and relabeling, there are {} nuclei left.'.format(n_left))

    return relabeled

In [ ]:
from scipy.ndimage import morphology

##Define the fillObjects 
def fillObjects(labeled_mask):
    
    #Let's do every z slice
    filled_mask=np.zeros_like(labeled_mask)
    for z, frame in enumerate(labeled_mask):

        #Could remove on boarders at this point?
        final=np.zeros_like(frame)

        for l in range(frame.max()):
            ll=l+1
            blob=morphology.binary_dilation(frame==ll, np.ones((20,20)))
            filled_blob=morphology.binary_fill_holes(blob)
            reduced=morphology.binary_erosion(filled_blob, np.ones((20,20)))*ll
            final=final+reduced

        filled_mask[z]=final
    return filled_mask

In [ ]:
mask6=fillObjects(relabeled)

In [ ]:
##Compare
plt.subplot(1,2,1),plt.imshow(mask5[z])
plt.subplot(1,2,1),plt.axis('off')
plt.subplot(1,2,1),plt.title('Mask')
plt.subplot(1,2,2),plt.imshow(mask6[z])
plt.subplot(1,2,2),plt.axis('off')
plt.subplot(1,2,2),plt.title('Filled Mask')

In [ ]:
##Could you watershed these as you go?

In [ ]:
##Calculate distance transform
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from scipy import ndimage
distance3D = ndimage.distance_transform_edt(mask6)

distance_z4 = ndimage.distance_transform_edt(mask6[4])

In [ ]:
##Compare distance vs 3D distance transform
plt.subplot(1,3,1),plt.imshow(mask6[z])
plt.subplot(1,3,1),plt.axis('off')
plt.subplot(1,3,1),plt.title('Labeled Mask')
plt.subplot(1,3,2),plt.imshow(distance_z4)
plt.subplot(1,3,2),plt.axis('off')
plt.subplot(1,3,2),plt.title('2D distance')
plt.subplot(1,3,3),plt.imshow(distance3D[z])
plt.subplot(1,3,3),plt.axis('off')
plt.subplot(1,3,3),plt.title('3D distance')

##3D transform better resolves the gaps

In [ ]:
#local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((1, 3, 3)), labels=mask6)
local_max1 = peak_local_max(distance_z4, indices=False)
local_max2 = peak_local_max(distance_z4, indices=False, labels=mask6[4])
local_max3 = peak_local_max(distance_z4, indices=False, threshold_abs=3, labels=mask6[4])
local_max4 = peak_local_max(distance_z4, indices=False, threshold_abs=3, footprint=np.ones((4, 4)), labels=mask6[4])

In [ ]:
#local_max5 = peak_local_max(distance_z4, indices=False, threshold_abs=3, footprint=np.ones((50, 50)), labels=mask6[4]) #50,50 works
local_max6 = peak_local_max(distance_z4, indices=False, threshold_abs=3, footprint=np.ones((75, 75)), labels=mask6[4]) #50,50 works

In [ ]:
plt.imshow(morphology.binary_dilation(local_max5, np.ones((5,5))), cmap='gray')
plt.axis('off')
plt.title('Max 1')

In [ ]:
##Compare distance vs 3D distance transform
plt.subplot(1,2,1),plt.imshow(mask6[z], cmap='gray')
plt.subplot(1,2,1),plt.axis('off')
plt.subplot(1,2,1),plt.title('Mask')
plt.subplot(1,2,2),plt.imshow(morphology.binary_dilation(local_max5, np.ones((5,5))), cmap='gray')
plt.subplot(1,2,2),plt.axis('off')
plt.subplot(1,2,2),plt.title('Max 5')
# plt.subplot(1,4,3),plt.imshow(local_max3, cmap='gray')
# plt.subplot(1,4,3),plt.axis('off')
# plt.subplot(1,4,3),plt.title('Max 3')
# plt.subplot(1,4,4),plt.imshow(local_max4, cmap='gray')
# plt.subplot(1,4,4),plt.axis('off')
# plt.subplot(1,4,4),plt.title('Max 4')

In [ ]:
##Check local max using overlay
plt.imshow(mh.overlay(mask6[z], morphology.binary_dilation(local_max6, np.ones((9,9)))))

In [ ]:
#Test 2D watershed
markers, n_max = mh.label(local_max6)
labels_ws = watershed(-distance_z4, markers, mask=mask6[z])

In [ ]:
plt.subplot(1,2,1),plt.imshow(mask6[z], cmap='gray')
plt.subplot(1,2,1),plt.axis('off')
plt.subplot(1,2,1),plt.title('Mask')
plt.subplot(1,2,2),plt.imshow(labels_ws, cmap='gray')
plt.subplot(1,2,2),plt.axis('off')
plt.subplot(1,2,2),plt.title('Watershed')

##Watershed did improve 2D but may split some nuclei

In [ ]:
##What about in 3D?
#Detect 2D maxima in a single plane of the 3D distance image
local_max7 = peak_local_max(distance3D[4], indices=False, threshold_abs=3, labels=mask6[4])

#Detect maxima in 3D space
local_max8 = peak_local_max(distance3D, indices=False, threshold_abs=3, labels=mask6)

In [ ]:
#Now try the improved 2D settings
local_max9 = peak_local_max(distance3D, indices=False, threshold_abs=3, footprint=np.ones((1, 75, 75)), labels=mask6)

In [ ]:
#Can you improve further?
local_max10 = peak_local_max(distance3D, indices=False, threshold_abs=5, footprint=np.ones((1, 100, 100)), labels=mask6)

In [ ]:
#Perhaps 2D peak detection within 3D distance map is best
local_max11 = peak_local_max(distance3D[4], indices=False, threshold_abs=5, footprint=np.ones((100, 100)), labels=mask6[4])
#This give identical peaks to local max10

In [ ]:
local_max10[4].all() == local_max11.all()

In [ ]:
##Compare 2D vs 3D local maxima
plt.subplot(1,2,1),plt.imshow(morphology.binary_dilation(local_max10[4], np.ones((5,5))), cmap='gray')
plt.subplot(1,2,1),plt.axis('off')
plt.subplot(1,2,1),plt.title('Max 10')
plt.subplot(1,2,2),plt.imshow(morphology.binary_dilation(local_max11, np.ones((5,5))), cmap='gray')
plt.subplot(1,2,2),plt.axis('off')
plt.subplot(1,2,2),plt.title('Max 11')

In [ ]:
##Check local max using overlay
plt.imshow(mh.overlay(mask6[z], morphology.binary_dilation(local_max10[z], np.ones((9,9)))))

In [ ]:
#Test 3D watershed in 2D
markers, n_max = mh.label(local_max10[4])
labels_ws2 = watershed(-distance3D[4], markers, mask=mask6[z])

In [ ]:
plt.subplot(1,2,1),plt.imshow(labels_ws, cmap='gray')
plt.subplot(1,2,1),plt.axis('off')
plt.subplot(1,2,1),plt.title('Mask')
plt.subplot(1,2,2),plt.imshow(labels_ws2, cmap='gray')
plt.subplot(1,2,2),plt.axis('off')
plt.subplot(1,2,2),plt.title('Watershed')

In [ ]:
##Increase the footprint even higher
local_max12 = peak_local_max(distance3D, indices=False, threshold_abs=7, footprint=np.ones((1, 175, 175)), labels=mask6)

In [ ]:
##Check local max using overlay
plt.imshow(mh.overlay(mask6[z], morphology.binary_dilation(local_max12[z], np.ones((9,9)))))

In [ ]:
markers, n_max = mh.label(local_max12)

In [ ]:
# ##Full 3D
# labels_ws3 = watershed(-distance3D, markers, mask=mask6)
# ##Full 3D sucks!!

In [ ]:
for f in range(24):
    ws4[f] = watershed(-distance3D[f], markers[f], mask=mask6[f])

In [ ]:
def watershed3D(mask, markers, distance):

    labels=np.zeros_like(mask)
    
    #Calculate labels in 2D
    for f, frame in enumerate(mask):
            
        labels[f] = watershed(-distance[f], markers[f], mask=frame)
        
    return labels

In [ ]:
##Iterative Watershed
ws4 = watershed3D(distance=distance3D, mask=mask6, markers=local_max12)

In [ ]:
plt.imshow(ws4[8])
#plt.imshow(watershed(-distance3D[4], markers[4], mask=mask6[4]))
#plt.imshow(watershed3D(distance=-distance3D[4], peaks=markers[4], mask=mask6[4]))

In [ ]:
##How do you separate distinct labels.
from scipy.ndimage import morphology
from skimage.morphology import disk

##Define the fillObjects 
def splitWatershed(watershed_labels):
    
    ##Add conditional to only separate objects within the right parameters
    
    radius = 3
    selem = disk(radius)
    
    #Let's do every z slice
    split=np.zeros_like(watershed_labels)
    for z, frame in enumerate(watershed_labels):
        print(z)

        #Could remove on boarders at this point?
        final=np.zeros_like(frame)

        for l in range(frame.max()):
            #print(l)
            ll=l+1
            shrunk=morphology.binary_erosion(frame==ll, selem)*ll
            final=final+shrunk
        split[z]=final
    return split

In [ ]:
#Try splitting the 3D watershed
split=splitWatershed(ws4)

In [ ]:
plt.imshow(split[4])

In [ ]:
#Then relabel the whole stack


In [ ]:
from skimage.morphology import disk

radius = 3
selem = disk(radius)

final=np.zeros_like(ws4[4])

for l in range(ws4[4].max()):
    ll=l+1
    shrunk=morphology.binary_erosion(ws4[4]==ll, selem)*ll
    final=final+shrunk

In [ ]:
split=np.zeros_like(ws4)
    for z, frame in enumerate(ws4):

        #Could remove on boarders at this point?
        final=np.zeros_like(frame)

        for l in range(frame.max()):
            ll=l+1
            shrunk=morphology.binary_erosion(frame==ll)*ll
            final=final+shrunk
    split[z]=final

In [ ]:
plt.subplot(1,2,1),plt.imshow(labels_ws, cmap='gray')
plt.subplot(1,2,1),plt.axis('off')
plt.subplot(1,2,1),plt.title('Mask')
plt.subplot(1,2,2),plt.imshow(labels_ws3[4], cmap='gray')
plt.subplot(1,2,2),plt.axis('off')
plt.subplot(1,2,2),plt.title('Watershed')

In [ ]:
from scipy.ndimage import morphology

##Define the fillObjects 
def fillAndWatershedObjects(labeled_mask):
    
    #Let's do every z slice
    filled_mask=np.zeros_like(labeled_mask)
    for z, frame in enumerate(labeled_mask):

        #Could remove on boarders at this point?
        final=np.zeros_like(frame)

        for l in range(frame.max()):
            ll=l+1
            blob=morphology.binary_dilation(frame==ll, np.ones((20,20)))
            filled_blob=morphology.binary_fill_holes(blob)
            
            ##Don't pass labels forward
            reduced=morphology.binary_erosion(filled_blob, np.ones((20,20)))
            final=final+reduced
        
        #In this script filled mask is binary
        filled_mask[z]=final
        
    #3D Watershed
    split_mask=ws(filled_mask)
        
    return split_mask

In [ ]:
##Compare
plt.subplot(1,3,1),plt.imshow(mask6[z])
plt.subplot(1,3,1),plt.axis('off')
plt.subplot(1,3,1),plt.title('Mask')
plt.subplot(1,3,2),plt.imshow(local_maxi[z])
plt.subplot(1,3,2),plt.axis('off')
plt.subplot(1,3,2),plt.title('Filled Mask')
plt.subplot(1,3,3),plt.imshow(labels_ws[z])
plt.subplot(1,3,3),plt.axis('off')
plt.subplot(1,3,3),plt.title('Filled Mask')

In [ ]:
#What?
plt.imshow(labels_ws[z])
plt.axis('off')
plt.title('Filled Mask')

In [ ]:
def ws(mask):
    
    from skimage.morphology import watershed
    from skimage.feature import peak_local_max
    from scipy import ndimage

    #This bit is simple
    distance = ndimage.distance_transform_edt(mask)

    #This bit is the challenge   
    local_maxi = peak_local_max(distance, 
                                indices=False,
                                footprint=np.ones((1, 25, 25)),
                                threshold_abs=3,
                                num_peaks=80, 
                                labels=mask)
    
    #Find the centers
    markers = morphology.label(local_maxi)
    
    #Watershed image
    labels_ws = watershed(-distance, markers, mask=mask6)
    
    return labels_ws

In [ ]:
new_mask = ws(mask6)

In [ ]:
plt.subplot(1,3,1),plt.imshow(mask6[z])
plt.subplot(1,3,1),plt.axis('off')
plt.subplot(1,3,1),plt.title('Mask')
plt.subplot(1,3,2),plt.imshow(distance[z])
plt.subplot(1,3,2),plt.axis('off')
plt.subplot(1,3,2),plt.title('Distance Transform')
plt.subplot(1,3,3),plt.imshow(new_mask[z])
plt.subplot(1,3,3),plt.axis('off')
plt.subplot(1,3,3),plt.title('Filled Mask')

In [ ]:
#This bit is the challenge, optimise the footprint

local_maxi_11 = peak_local_max(distance, 
                               threshold_abs=3, #Peak pixels must be more than 3 pixels from the background
                               indices=False, 
                               footprint=np.ones((1, 11, 11)), #A difficult parameter to set
                               labels=mask6,  #I think this will prevent peaks outside of labels
                               num_peaks=80)  #set this to double the number of objects.

#local_maxi_11 = peak_local_max(distance, indices=False, footprint=np.ones((1, 11, 11)), labels=mask6)

In [ ]:
#How appropriate is the local max detection
plt.subplot(1,3,1),plt.imshow(distance[z], cmap='gray')
plt.subplot(1,3,1),plt.axis('off')
plt.subplot(1,3,1),plt.title('Distance')
plt.subplot(1,3,2),plt.imshow(local_maxi[z], cmap='gray')
plt.subplot(1,3,2),plt.axis('off')
plt.subplot(1,3,2),plt.title('Local Max 3x3')
plt.subplot(1,3,3),plt.imshow(local_maxi_11[z], cmap='gray')
plt.subplot(1,3,3),plt.axis('off')
plt.subplot(1,3,3),plt.title('Local Max 11x11')

In [ ]:
plt.imshow(local_maxi_11[z])
plt.axis('off')
plt.title('Local Max 11x11')

In [ ]:
#Find the centers
markers = morphology.label(local_maxi_11)

#Watershed image
labels_ws = watershed(-distance, markers, mask=mask6)

In [ ]:
plt.imshow(labels_ws[z])
plt.axis('off')
plt.title('Watershed')

In [ ]:
splitMask=fillAndWatershedObjects(mask6)

In [ ]:
plt.imshow(splitMask[z])
plt.axis('off')
plt.title('Watershed')